In [1]:
# Import necessary libraries
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
#from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Load the dataset
#df = pd.read_csv("PERC_mendelly.csv")
#df = pd.read_csv("poem.csv")
df = pd.read_csv("final.csv")
df = df.dropna()
# Preprocess the text
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

df['clean_text'] = df['Poem'].apply(preprocess)

# Split the poem column into individual stanzas
df['stanzas'] = df['Poem'].apply(sent_tokenize)

# Convert emotions column to numerical labels
emotions = ['love', 'sad', 'anger', 'hate', 'fear', 'surprise', 'courage', 'joy', 'peace',"hope",'care']
df['emotion_label'] = df['Emotion'].apply(emotions.index)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['stanzas'], df['emotion_label'], test_size=0.2, random_state=42)

# Extract features from the stanzas using TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
#vectorizer = CountVectorizer()
X_train_tfidf = vectorizer.fit_transform([' '.join(stanza) for stanza in X_train])
X_test_tfidf = vectorizer.transform([' '.join(stanza) for stanza in X_test])

# Train a logistic regression model
model = LogisticRegression(random_state=42,max_iter=1000)
model.fit(X_train_tfidf, y_train)

# Evaluate the performance of the model
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100,"%")

# Use the trained model to predict the emotion of each stanza in a new poem
new_poem = """
My mother bore me in the southern wild,
And I am black, but O! my soul is white;
White as an angel is the English child: 
But I am black as if bereav'd of light.

My mother taught me underneath a tree 
And sitting down before the heat of day,
She took me on her lap and kissed me,
And pointing to the east began to say. 

Look on the rising sun: there God does live 
And gives his light, and gives his heat away. 
And flowers and trees and beasts and men receive
Comfort in morning joy in the noonday.

And we are put on earth a little space,
That we may learn to bear the beams of love, 
And these black bodies and this sun-burnt face
Is but a cloud, and like a shady grove.

For when our souls have learn'd the heat to bear 
The cloud will vanish we shall hear his voice. 
Saying: come out from the grove my love & care,
And round my golden tent like lambs rejoice.

Thus did my mother say and kissed me, 
And thus I say to little English boy. 
When I from black and he from white cloud free,
And round the tent of God like lambs we joy: 

Ill shade him from the heat till he can bear, 
To lean in joy upon our fathers knee. 
And then I'll stand and stroke his silver hair,
And be like him and he will then love me.
"""
new_stanzas = sent_tokenize(new_poem)

for stanza in new_stanzas:
    lines = stanza.strip().split('\n')
    line_emotions = []
    for line in lines:
        line_tfidf = vectorizer.transform([preprocess(line)])
        line_emotion = emotions[model.predict(line_tfidf)[0]]
        line_emotions.append(line_emotion)
    print('Stanza:')
    print(stanza)
    print('Emotions:')
    print(line_emotions)
    print("\n")

Accuracy: 33.56164383561644 %
Stanza:

My mother bore me in the southern wild,
And I am black, but O!
Emotions:
['sad', 'sad']


Stanza:
my soul is white;
White as an angel is the English child: 
But I am black as if bereav'd of light.
Emotions:
['sad', 'sad', 'hope']


Stanza:
My mother taught me underneath a tree 
And sitting down before the heat of day,
She took me on her lap and kissed me,
And pointing to the east began to say.
Emotions:
['hope', 'hope', 'hope', 'love']


Stanza:
Look on the rising sun: there God does live 
And gives his light, and gives his heat away.
Emotions:
['hope', 'hope']


Stanza:
And flowers and trees and beasts and men receive
Comfort in morning joy in the noonday.
Emotions:
['hope', 'hope']


Stanza:
And we are put on earth a little space,
That we may learn to bear the beams of love, 
And these black bodies and this sun-burnt face
Is but a cloud, and like a shady grove.
Emotions:
['sad', 'love', 'sad', 'hope']


Stanza:
For when our souls have learn'd th

In [116]:
df

,Poem,Emotion,clean_text,stanzas,emotion_label
0,"When I look into your eyes,\nI see a thousand ...",love,"look eye , see thousand star sky . 're smile f...","[When I look into your eyes,\nI see a thousand...",0
1,"When you hold me in your arms,\nI can't resist...",love,"hold arm , ca n't resist charm . day okay , tr...","[When you hold me in your arms,\nI can't resis...",0
2,"When I feel the weight of the world,\nYou're t...",love,"feel weight world , 're one need . 're world ,...","[When I feel the weight of the world,\nYou're ...",0
3,"When I wake up this morning,\nThe sun is shinn...",love,"wake morn , sun shin , bird sing , face smile ...","[When I wake up this morning,\nThe sun is shin...",0
4,But I'm feeling this is love\n'Cause I have be...,love,'m feel love 'caus way . give good reason live...,[But I'm feeling this is love\n'Cause I have b...,0
...,...,...,...,...,...
513,"I've a wife I love, and that loves me;\nI've a...",love,"'ve wife love , love ; 've rich bodili .","[I've a wife I love, and that loves me;\nI've ...",0
514,"I am in God's presence night and day,\nAnd He ...",fear,"god 's presenc night day , never turn face awa...","[I am in God's presence night and day,\nAnd He...",4
515,"For my worldly things God makes him pay,\nAnd ...",care,"worldli thing god make pay , 'd pay would pray...","[For my worldly things God makes him pay,\nAnd...",10
516,"Then if for riches I must not pray,\nGod knows...",joy,"rich must pray , god know , littl prayer need ...","[Then if for riches I must not pray,\nGod know...",7


In [2]:
# Import necessary libraries
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Load the dataset
#df = pd.read_csv("PERC_mendelly.csv")
#df = pd.read_csv("poem.csv")
df = pd.read_csv("final_trial.csv")
df=df.dropna()
# Preprocess the text
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

df['clean_text'] = df['Poem'].apply(preprocess)

# Split the poem column into individual stanzas
df['stanzas'] = df['Poem'].apply(sent_tokenize)

# Convert emotions column to numerical labels
emotions = ['love', 'sad', 'anger', 'hate', 'fear', 'surprise', 'courage', 'joy', 'peace',"hope",'care']
df['emotion_label'] = df['Emotion'].apply(emotions.index)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['stanzas'], df['emotion_label'], test_size=0.2, random_state=42)

# Extract features from the stanzas using TfidfVectorizer
#vectorizer = TfidfVectorizer(stop_words='english')
vectorizer = CountVectorizer()
X_train_tfidf = vectorizer.fit_transform([' '.join(stanza) for stanza in X_train])
X_test_tfidf = vectorizer.transform([' '.join(stanza) for stanza in X_test])

# Train a Naive Bayes model
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Evaluate the performance of the model
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100,"%")

# Use the trained model to predict the emotion of each stanza in a new poem
new_poem="""
Tyger Tyger, burning bright,
In the forests of the night;
What immortal hand or eye,
Could frame thy fearful symmetry?

In what distant deeps or skies.
Burnt the fire of thine eyes?
On what wings dare he aspire?
What the hand, dare seize the fire?

And what shoulder, & what art,
Could twist the sinews of thy heart?
And when thy heart began to beat,
What dread hand? & what dread feet?

What the hammer? what the chain,
In what furnace was thy brain?
What the anvil? what dread grasp,
Dare its deadly terrors clasp!

When the stars threw down their spears
And water’d heaven with their tears:
Did he smile his work to see?
Did he who made the Lamb make thee?

Tyger Tyger burning bright,
In the forests of the night:
What immortal hand or eye,
Dare frame thy fearful symmetry?
"""
new_stanzas = sent_tokenize(new_poem)

for stanza in new_stanzas:
    lines = stanza.strip().split('\n')
    line_emotions = []
    for line in lines:
        line_tfidf = vectorizer.transform([preprocess(line)])
        line_emotion = emotions[model.predict(line_tfidf)[0]]
        line_emotions.append(line_emotion)
    print('Stanza:')
    print(stanza)
    print('Emotions:')
    print(line_emotions)
    print("\n")

Accuracy: 37.37474949899799 %
Stanza:

Tyger Tyger, burning bright,
In the forests of the night;
What immortal hand or eye,
Could frame thy fearful symmetry?
Emotions:
['surprise', 'joy', 'joy', 'joy']


Stanza:
In what distant deeps or skies.
Emotions:
['sad']


Stanza:
Burnt the fire of thine eyes?
Emotions:
['joy']


Stanza:
On what wings dare he aspire?
Emotions:
['sad']


Stanza:
What the hand, dare seize the fire?
Emotions:
['sad']


Stanza:
And what shoulder, & what art,
Could twist the sinews of thy heart?
Emotions:
['love', 'love']


Stanza:
And when thy heart began to beat,
What dread hand?
Emotions:
['love', 'sad']


Stanza:
& what dread feet?
Emotions:
['joy']


Stanza:
What the hammer?
Emotions:
['love']


Stanza:
what the chain,
In what furnace was thy brain?
Emotions:
['sad', 'sad']


Stanza:
What the anvil?
Emotions:
['love']


Stanza:
what dread grasp,
Dare its deadly terrors clasp!
Emotions:
['sad', 'sad']


Stanza:
When the stars threw down their spears
And water’d h

In [118]:
df

,Poem,Emotion,clean_text,stanzas,emotion_label
0,"When I look into your eyes,\nI see a thousand ...",love,"look eye , see thousand star sky . 're smile f...","[When I look into your eyes,\nI see a thousand...",0
1,"When you hold me in your arms,\nI can't resist...",love,"hold arm , ca n't resist charm . day okay , tr...","[When you hold me in your arms,\nI can't resis...",0
2,"When I feel the weight of the world,\nYou're t...",love,"feel weight world , 're one need . 're world ,...","[When I feel the weight of the world,\nYou're ...",0
3,"When I wake up this morning,\nThe sun is shinn...",love,"wake morn , sun shin , bird sing , face smile ...","[When I wake up this morning,\nThe sun is shin...",0
4,But I'm feeling this is love\n'Cause I have be...,love,'m feel love 'caus way . give good reason live...,[But I'm feeling this is love\n'Cause I have b...,0
...,...,...,...,...,...
513,"I've a wife I love, and that loves me;\nI've a...",love,"'ve wife love , love ; 've rich bodili .","[I've a wife I love, and that loves me;\nI've ...",0
514,"I am in God's presence night and day,\nAnd He ...",fear,"god 's presenc night day , never turn face awa...","[I am in God's presence night and day,\nAnd He...",4
515,"For my worldly things God makes him pay,\nAnd ...",care,"worldli thing god make pay , 'd pay would pray...","[For my worldly things God makes him pay,\nAnd...",10
516,"Then if for riches I must not pray,\nGod knows...",joy,"rich must pray , god know , littl prayer need ...","[Then if for riches I must not pray,\nGod know...",7


In [3]:
# Import necessary libraries
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the dataset
#df = pd.read_csv("PERC_mendelly.csv")
#df = pd.read_csv("poem.csv")
df = pd.read_csv("final_trial.csv")
df=df.dropna()
# Preprocess the text
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

df['clean_text'] = df['Poem'].apply(preprocess)

# Split the poem column into individual stanzas
df['stanzas'] = df['Poem'].apply(sent_tokenize)

# Convert emotions column to numerical labels
emotions = ['love', 'sad', 'anger', 'hate', 'fear', 'surprise', 'courage', 'joy', 'peace',"hope",'care']
df['emotion_label'] = df['Emotion'].apply(emotions.index)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['stanzas'], df['emotion_label'], test_size=0.2, random_state=42)

# Extract features from the stanzas using TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
#vectorizer = CountVectorizer()
X_train_tfidf = vectorizer.fit_transform([' '.join(stanza) for stanza in X_train])
X_test_tfidf = vectorizer.transform([' '.join(stanza) for stanza in X_test])

# Train an SVM model
model = SVC(random_state=42)
model.fit(X_train_tfidf, y_train)

# Evaluate the performance of the model
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100,"%")

# Use the trained model to predict the emotion of each stanza in a new poem
new_poem="""
Weavers, weaving at break of day,
Why do you weave a garment so gay?
Blue as the wing of a halcyon wild,
We weave the robes of a new-born child.

Weavers, weaving at fall of night,
Why do you weave a garment so bright?
Like the plumes of a peacock, purple and green,
We weave the marriage-veils of a queen.

Weavers, weaving solemn and still,
What do you weave in the moonlight chill? 
White as a feather and white as a cloud,
We weave a dead man's funeral shroud.
"""
new_stanzas = sent_tokenize(new_poem)

for stanza in new_stanzas:
    lines = stanza.strip().split('\n')
    line_emotions = []
    for line in lines:
        line_tfidf = vectorizer.transform([preprocess(line)])
        line_emotion = emotions[model.predict(line_tfidf)[0]]
        line_emotions.append(line_emotion)
    print('Stanza:')
    print(stanza)
    print('Emotions:')
    print(line_emotions)
    print("\n")

Accuracy: 38.476953907815634 %
Stanza:

Weavers, weaving at break of day,
Why do you weave a garment so gay?
Emotions:
['love', 'joy']


Stanza:
Blue as the wing of a halcyon wild,
We weave the robes of a new-born child.
Emotions:
['joy', 'sad']


Stanza:
Weavers, weaving at fall of night,
Why do you weave a garment so bright?
Emotions:
['love', 'joy']


Stanza:
Like the plumes of a peacock, purple and green,
We weave the marriage-veils of a queen.
Emotions:
['joy', 'joy']


Stanza:
Weavers, weaving solemn and still,
What do you weave in the moonlight chill?
Emotions:
['sad', 'sad']


Stanza:
White as a feather and white as a cloud,
We weave a dead man's funeral shroud.
Emotions:
['sad', 'sad']




In [120]:
import pandas as pd

# Load the dataset
df = pd.read_csv("PERC_mendelly.csv")

# Define a function to split the poem into stanzas
def split_poem(poem):
    return poem.split("\n\n")

# Apply the function to the 'poem' column and create a new 'stanza' column
df['Stanza'] = df['Poem'].apply(split_poem)

# Explode the 'stanza' column to create a new row for each stanza
df = df.explode('Stanza')

# Save the result to a new CSV file
df.to_csv("PERC_mendelly_stanzas.csv", index=False)


In [68]:
df = pd.read_csv("PERC_mendelly_stanzas.csv")
df = df.drop('Unnamed: 2',axis=1)

KeyError: "['Unnamed: 2'] not found in axis"

In [ ]:
df

In [69]:
df.isna().sum()

Poem         0
Emotion      0
Stanza     174
dtype: int64

In [70]:
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [71]:
df.isna().sum()

Poem       0
Emotion    0
Stanza     0
dtype: int64

In [72]:
df.shape

(4061, 3)

In [73]:
df.head(20)

,Poem,Emotion,Stanza
0,A Tree\nA tree beside the sandy river-beach \n...,sad,A Tree\nA tree beside the sandy river-beach \n...
1,A Tree\nA tree beside the sandy river-beach \n...,sad,This is the soul of man. Body and brain \nHung...
2,"Sri Krishna\n\nO immense Light and thou, O spi...",love,Sri Krishna
3,"Sri Krishna\n\nO immense Light and thou, O spi...",love,"O immense Light and thou, O spirit-wide boundl..."
4,"Who\n\n\nIn the blue of the sky, in the green ...",peace,Who
5,"Who\n\n\nIn the blue of the sky, in the green ...",peace,"\nIn the blue of the sky, in the green of the ..."
6,"Who\n\n\nIn the blue of the sky, in the green ...",peace,"He is lost in the heart, in the cavern of Natu..."
7,"Who\n\n\nIn the blue of the sky, in the green ...",peace,"In the strength of a man, in the beauty of wom..."
8,"Who\n\n\nIn the blue of the sky, in the green ...",peace,These are His works and His veils and His shad...
9,"Who\n\n\nIn the blue of the sky, in the green ...",peace,We have love for a boy who is dark and resplen...


In [74]:
df.to_csv("PERC_mendelly_stanzas_cleaned.csv", index=False)

In [2]:
# Import necessary libraries
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_csv("PERC_mendelly.csv")

#df = pd.read_csv("final_trial.csv")
df = df.dropna()

# Preprocess the text
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

df['clean_text'] = df['Poem'].apply(preprocess)

# Split the poem column into individual stanzas
df['stanzas'] = df['Poem'].apply(sent_tokenize)

# Convert emotions column to numerical labels
emotions = ['love', 'sad', 'anger', 'hate', 'fear', 'surprise', 'courage', 'joy', 'peace',"hope",'care']
df['emotion_label'] = df['Emotion'].apply(emotions.index)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['stanzas'], df['emotion_label'], test_size=0.2, random_state=42)

# Extract features from the stanzas using TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform([' '.join(stanza) for stanza in X_train])
X_test_tfidf = vectorizer.transform([' '.join(stanza) for stanza in X_test])
import joblib
joblib.dump(vectorizer, 'vectorizer.pkl')
# Train a Passive Aggressive Classifier
model = PassiveAggressiveClassifier(C=1.0, max_iter=1000, random_state=42)
model.fit(X_train_tfidf, y_train)

# Evaluate the performance of the model
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100,"%")

# Use the trained model to predict the emotion of each stanza in a new poem
new_poem = """
Weavers, weaving at break of day,
Why do you weave a garment so gay?
Blue as the wing of a halcyon wild,
We weave the robes of a new-born child.

Weavers, weaving at fall of night,
Why do you weave a garment so bright?
Like the plumes of a peacock, purple and green,
We weave the marriage-veils of a queen.

Weavers, weaving solemn and still,
What do you weave in the moonlight chill? 
White as a feather and white as a cloud,
We weave a dead man's funeral shroud.
"""
new_stanzas = sent_tokenize(new_poem)

for stanza in new_stanzas:
    lines = stanza.strip().split('\n')
    line_emotions = []
    for line in lines:
        line_tfidf = vectorizer.transform([preprocess(line)])
        line_emotion = emotions[model.predict(line_tfidf)[0]]
        line_emotions.append(line_emotion)
    print('Stanza:')
    print(stanza)
    print('Emotions:')
    print(line_emotions)
    print("\n")


Accuracy: 45.20547945205479 %
Stanza:

Weavers, weaving at break of day,
Why do you weave a garment so gay?
Emotions:
['love', 'joy']


Stanza:
Blue as the wing of a halcyon wild,
We weave the robes of a new-born child.
Emotions:
['joy', 'joy']


Stanza:
Weavers, weaving at fall of night,
Why do you weave a garment so bright?
Emotions:
['love', 'joy']


Stanza:
Like the plumes of a peacock, purple and green,
We weave the marriage-veils of a queen.
Emotions:
['joy', 'joy']


Stanza:
Weavers, weaving solemn and still,
What do you weave in the moonlight chill?
Emotions:
['peace', 'peace']


Stanza:
White as a feather and white as a cloud,
We weave a dead man's funeral shroud.
Emotions:
['peace', 'sad']




In [4]:
# Import necessary libraries
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score

# Load the dataset
df = pd.read_csv("final_trial.csv")

# Preprocess the text
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token not in stop_words]
    tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(tokens)

df['clean_text'] = df['Poem'].apply(preprocess)

# Split the poem column into individual stanzas
df['stanzas'] = df['Poem'].apply(sent_tokenize)

# Convert emotions column to numerical labels
emotions = ['love', 'sad', 'anger', 'hate', 'fear', 'surprise', 'courage', 'joy', 'peace',"hope","care"]
df['emotion_label'] = df['Emotion'].apply(emotions.index)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['stanzas'], df['emotion_label'], test_size=0.2, random_state=42)

# Extract features from the stanzas using TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')
X_train_tfidf = vectorizer.fit_transform([' '.join(stanza) for stanza in X_train])
X_test_tfidf = vectorizer.transform([' '.join(stanza) for stanza in X_test])

# Train a passive aggressive classifier model
model = PassiveAggressiveClassifier(random_state=42,max_iter=100)
model.fit(X_train_tfidf, y_train)

# Evaluate the performance of the model
y_pred = model.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy*100,"%")

# Use the trained model to predict the emotion of each stanza in a new poem
new_poem = """
My mother bore me in the southern wild,
And I am black, but O! my soul is white;
White as an angel is the English child: 
But I am black as if bereav'd of light.

My mother taught me underneath a tree 
And sitting down before the heat of day,
She took me on her lap and kissed me,
And pointing to the east began to say. 

Look on the rising sun: there God does live 
And gives his light, and gives his heat away. 
And flowers and trees and beasts and men receive
Comfort in morning joy in the noonday.

And we are put on earth a little space,
That we may learn to bear the beams of love, 
And these black bodies and this sun-burnt face
Is but a cloud, and like a shady grove.

For when our souls have learn'd the heat to bear 
The cloud will vanish we shall hear his voice. 
Saying: come out from the grove my love & care,
And round my golden tent like lambs rejoice.

Thus did my mother say and kissed me, 
And thus I say to little English boy. 
When I from black and he from white cloud free,
And round the tent of God like lambs we joy: 

Ill shade him from the heat till he can bear, 
To lean in joy upon our fathers knee. 
And then I'll stand and stroke his silver hair,
And be like him and he will then love me.
"""
new_stanzas = sent_tokenize(new_poem)
new_stanzas_tfidf = vectorizer.transform(new_stanzas)
new_stanzas_emotions = model.predict(new_stanzas_tfidf)
new_stanzas_emotions = [emotions[label] for label in new_stanzas_emotions]
for stanza, emotion in zip(new_stanzas, new_stanzas_emotions):
    print('stanzas:', stanza.strip())
    print('Emotion:', emotion)
    print("\n")

Accuracy: 37.17434869739479 %
stanzas: My mother bore me in the southern wild,
And I am black, but O!
Emotion: sad


stanzas: my soul is white;
White as an angel is the English child: 
But I am black as if bereav'd of light.
Emotion: sad


stanzas: My mother taught me underneath a tree 
And sitting down before the heat of day,
She took me on her lap and kissed me,
And pointing to the east began to say.
Emotion: love


stanzas: Look on the rising sun: there God does live 
And gives his light, and gives his heat away.
Emotion: courage


stanzas: And flowers and trees and beasts and men receive
Comfort in morning joy in the noonday.
Emotion: hope


stanzas: And we are put on earth a little space,
That we may learn to bear the beams of love, 
And these black bodies and this sun-burnt face
Is but a cloud, and like a shady grove.
Emotion: sad


stanzas: For when our souls have learn'd the heat to bear 
The cloud will vanish we shall hear his voice.
Emotion: hope


stanzas: Saying: come out f